In [72]:
'''
This notebook combines the fangraphs advanced and standard pages, and then merges this (inner join) on
the ESPN free agent data. We then go through many of the columns and massage/clean our data
'''

'\nThis notebook combines the fangraphs advanced and standard pages, and then merges this (inner join) on\nthe ESPN free agent data. We then go through many of the columns and massage/clean our data\n'

In [73]:
import pickle
import pandas as pd
import numpy as np
import copy
import seaborn as sns

In [74]:
advanced_df = pickle.load( open( "df_advanced.pkl", "rb" ) )
standard_df = pickle.load( open( "df_standard.pkl", "rb" ) )
free_agents_df = pickle.load( open( "free_agents_master.pkl", "rb" ) )

In [75]:
advanced_df.head()
advanced_df.count()

PA       3567
BB%      3567
K%       3567
BB/K     3567
AVG      3567
OBP      3567
SLG      3567
OPS      3567
ISO      3567
Spd      3567
BABIP    3567
UBR      3567
wGDP     3567
wSB      3567
wRC      3567
wRAA     3567
wOBA     3567
wRC+     3567
dtype: int64

In [76]:
standard_df.head()
standard_df.count()

G      3567
AB     3567
PA     3567
H      3567
1B     3567
2B     3567
3B     3567
HR     3567
R      3567
RBI    3567
BB     3567
IBB    3567
SO     3567
HBP    3567
SF     3567
SH     3567
GDP    3567
SB     3567
CS     3567
AVG    3567
dtype: int64

In [77]:
#Joining on the index for both columns
#master_df becomes our primary dataframe
master_df = pd.merge(advanced_df, standard_df, how='inner',left_index=True,right_index=True)

In [78]:
master_df.count().head() #3567 - our merge worked

PA_x     3567
BB%      3567
K%       3567
BB/K     3567
AVG_x    3567
dtype: int64

In [79]:
#Save progress
with open('df_standard_advanced.pkl', 'wb') as picklefile:
    pickle.dump(master_df, picklefile)

In [80]:
#Now, let's join on the Free Agent Data we have - should be around 600 names left?

In [81]:
free_agents_df.groupby('PLAYER_YEAR').count().head()
free_agents_df = free_agents_df.drop_duplicates() #some dupes - easy enough to drop
free_agents_df.groupby('PLAYER_YEAR').count().head() #all 1's

,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS
PLAYER_YEAR,,,,,,,,,
A.J. Achter 2016,1,1,1,1,1,1,1,1,1
A.J. Burnett 2013,1,1,1,1,1,1,1,1,1
A.J. Burnett 2014,1,1,1,1,1,1,1,1,1
A.J. Burnett 2015,1,1,1,1,1,1,1,1,1
A.J. Ellis 2016,1,1,1,1,1,1,1,1,1


In [82]:
#merge master with free_agents_df - our ESPN data
master_df = pd.merge(master_df,free_agents_df,how='inner',\
                     left_index=True,right_on="PLAYER_YEAR")

In [83]:
master_df.describe() #534 names - that sounds about right

,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,Spd,...,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,PLAYER_YEAR
count,534,534,534,534,534,534,534,534,534,534,...,534,534,534,534,534,534,534,534,534,534
unique,346,132,229,94,141,152,236,291,209,79,...,326,11,25,5,30,32,11,51,143,534
top,236,7.9 %,20.5 %,0.49,.246,.314,.327,.713,.098,2.1,...,A.J. Pierzynski,LF,34,Signed,Orioles,Yankees,1,NR,Minor Lg,Chris Carter 2010
freq,5,12,8,19,14,10,7,6,8,18,...,5,101,52,450,26,25,232,362,135,1


In [84]:
#PLAYER_YEAR can be our index - easy to see what the row represents
master_df.set_index(["PLAYER_YEAR"],drop=False,inplace=True) 

In [85]:
master_df.head()

,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,Spd,...,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,PLAYER_YEAR
PLAYER_YEAR,,,,,,,,,,,,,,,,,,,,,
Jim Thome 2010,340,17.6 %,24.1 %,0.73,.283,.412,.627,1.039,.344,2.3,...,Jim Thome,DH,47,Signed,Twins,Twins,1,24,"$3,000,000",Jim Thome 2010
J.D. Martinez 2017,489,10.8 %,26.2 %,0.41,.303,.376,.690,1.066,.387,4.7,...,J.D. Martinez,RF,30,Signed,Diamondbacks,Red Sox,5,6,"$110,000,000",J.D. Martinez 2017
Victor Martinez 2014,641,10.9 %,6.6 %,1.67,.335,.409,.565,.974,.230,2.0,...,Victor Martinez,DH,39,Signed,Tigers,Tigers,4,12,"$68,000,000",Victor Martinez 2014
Franklin Gutierrez 2015,189,7.4 %,28.6 %,0.26,.292,.354,.620,.974,.327,1.0,...,Franklin Gutierrez,LF,35,Signed,Mariners,Mariners,1,NR,--,Franklin Gutierrez 2015
Paul Konerko 2010,631,11.4 %,17.4 %,0.65,.312,.393,.584,.977,.272,1.7,...,Paul Konerko,DH,42,Signed (A),White Sox,White Sox,3,12,"$37,500,000",Paul Konerko 2010


In [86]:
#Save progress
with open('master_df.pkl', 'wb') as picklefile:
    pickle.dump(master_df, picklefile)

## Data Cleaning Time

In [87]:
#BB%, K% have percentage signs
master_df[["BB%","K%"]].head()

,BB%,K%
PLAYER_YEAR,,
Jim Thome 2010,17.6 %,24.1 %
J.D. Martinez 2017,10.8 %,26.2 %
Victor Martinez 2014,10.9 %,6.6 %
Franklin Gutierrez 2015,7.4 %,28.6 %
Paul Konerko 2010,11.4 %,17.4 %


In [88]:
#Replace % with empty
for col in ["BB%","K%"]:
    master_df[col] = master_df[col].str.replace(" \%","")

In [89]:
#Replace $ or Commas in DOLLARS with blank
master_df["DOLLARS"] = master_df["DOLLARS"].str.replace("\$|,","")

In [90]:
#362 of the 500+ names have reported MLB salaries.
master_df[master_df["DOLLARS"].str.isnumeric()] 
#I will set the Minor League salary to the league minimum

,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,Spd,...,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,PLAYER_YEAR
PLAYER_YEAR,,,,,,,,,,,,,,,,,,,,,
Jim Thome 2010,340,17.6,24.1,0.73,.283,.412,.627,1.039,.344,2.3,...,Jim Thome,DH,47,Signed,Twins,Twins,1,24,3000000,Jim Thome 2010
J.D. Martinez 2017,489,10.8,26.2,0.41,.303,.376,.690,1.066,.387,4.7,...,J.D. Martinez,RF,30,Signed,Diamondbacks,Red Sox,5,6,110000000,J.D. Martinez 2017
Victor Martinez 2014,641,10.9,6.6,1.67,.335,.409,.565,.974,.230,2.0,...,Victor Martinez,DH,39,Signed,Tigers,Tigers,4,12,68000000,Victor Martinez 2014
Paul Konerko 2010,631,11.4,17.4,0.65,.312,.393,.584,.977,.272,1.7,...,Paul Konerko,DH,42,Signed (A),White Sox,White Sox,3,12,37500000,Paul Konerko 2010
Michael Cuddyer 2014,205,6.8,14.6,0.47,.332,.376,.579,.955,.247,5.0,...,Michael Cuddyer,LF,39,Signed,Rockies,Mets,2,46,21000000,Michael Cuddyer 2014
Prince Fielder 2011,692,15.5,15.3,1.01,.299,.415,.566,.981,.267,1.9,...,Prince Fielder,DH,33,Signed (A),Brewers,Tigers,9,1,214000000,Prince Fielder 2011
Jayson Werth 2010,652,12.6,22.5,0.56,.296,.388,.532,.921,.236,4.9,...,Jayson Werth,LF,38,Signed (A),Phillies,Nationals,7,3,126000000,Jayson Werth 2010
Shin-Soo Choo 2013,712,15.7,18.7,0.84,.285,.423,.462,.885,.178,4.6,...,Shin-Soo Choo,RF,35,Signed,Reds,Rangers,7,5,130000000,Shin-Soo Choo 2013
Zack Cozart 2017,507,12.2,15.4,0.79,.297,.385,.548,.933,.251,5.3,...,Zack Cozart,3B,32,Signed,Reds,Angels,3,3,38000000,Zack Cozart 2017


In [91]:
#135 guys signed to minor league deals - we'll give them the League Minimum salary
minimum_players = list(master_df[master_df.DOLLARS=="Minor Lg"]["PLAYER_YEAR"]) 

https://www.statista.com/statistics/256187/minimum-salary-of-players-in-major-league-baseball/  
Average Minimum Salary of MLB players from 2011-2018 is around $497000 - we'll use that as our value for these guys

In [92]:
#Some of these guys have YRS missing as well - we're assuming they signed 1 year, league minimum contracts
yrs_missing = list(master_df[master_df["YRS"].isnull()].index)

In [93]:
'''
For these players - set dollars to 497K, YRS to 1
'''
for name in minimum_players:
    master_df.loc[master_df.PLAYER_YEAR==name,"DOLLARS"] = 497000

for name in yrs_missing:
    master_df.loc[master_df.PLAYER_YEAR==name,"YRS"] = 1

In [94]:
#Players Signed to Major League deals without salaries/yrs reported
#These were grabbed from Baseball Reference
#Players were given NaN if I couldnt find their contract value
fill_in_salaries = {'Alexi Amarista 2016':1100000,
 'Arismendy Alcantara 2017':np.NaN,
 'Clint Barmes 2013':2000000,
 'David Adams 2013':np.NaN,
 'Franklin Gutierrez 2015':1500000,
 'Gerald Laird 2012':3000000,
 'Hyun Soo Kim 2017':np.NaN,
 'Jarrod Saltalamacchia 2015':507500,
 'Jason Bay 2012':1000000,
 'Luke Scott 2013':np.NaN,
 'Lyle Overbay 2011':1000000,
 'Ronny Cedeno 2012':1150000,
 'Ronny Cedeno 2011':1200000,
 'Yuniesky Betancourt 2013':np.NaN,
 'Jon Jay 2017':3000000,
 'Orlando Cabrera 2010':1000000,
 'Ryan Ludwick 2011':2000000,
 'Scott Hairston 2012':2500000,
 'Travis Hafner 2012':2000000}

In [95]:
#Use fill_in_salaries to update those players
for name,amount in fill_in_salaries.items():
#     print(f"Name: {name}")
    master_df.loc[master_df.PLAYER_YEAR==name,"DOLLARS"] = amount
#     print(master_df.loc[name]["DOLLARS"])
    

In [96]:
# This should be empty now - any signed players now have data (or an NaN, if we couldnt find anything)
master_df[(master_df["DOLLARS"]=="--")&(master_df["STATUS"]=="Signed")].sort_values(by="PLAYER")

,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,Spd,...,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,PLAYER_YEAR
PLAYER_YEAR,,,,,,,,,,,,,,,,,,,,,


In [97]:
#Last - we want to drop retired players
retired_players = master_df[(master_df["DOLLARS"]=="--")].sort_values(by="PLAYER") 
retired_players.head(10)

,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,Spd,...,PLAYER,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,PLAYER_YEAR
PLAYER_YEAR,,,,,,,,,,,,,,,,,,,,,
Bobby Abreu 2014,155,12.9,13.5,0.95,.248,.342,.338,.680,.090,2.4,...,Bobby Abreu,RF,44,Retired,Mets,--,,NR,--,Bobby Abreu 2014
Chipper Jones 2012,448,12.7,11.4,1.12,.287,.377,.455,.832,.168,2.1,...,Chipper Jones,3B,45,Retired,Braves,--,,NR,--,Chipper Jones 2012
Craig Counsell 2011,187,10.7,11.2,0.95,.178,.280,.223,.503,.045,4.5,...,Craig Counsell,2B,47,Retired,Brewers,--,,NR,--,Craig Counsell 2011
David Ross 2016,205,14.6,26.3,0.56,.229,.338,.446,.784,.217,1.4,...,David Ross,C,41,Retired,Cubs,--,,NR,--,David Ross 2016
Derek Jeter 2014,634,5.5,13.7,0.40,.256,.304,.313,.617,.057,3.5,...,Derek Jeter,SS,43,Retired,Yankees,--,,NR,--,Derek Jeter 2014
Jorge Posada 2011,387,10.1,19.6,0.51,.235,.315,.398,.714,.163,1.1,...,Jorge Posada,DH,46,Retired,Yankees,--,,NR,--,Jorge Posada 2011
Josh Willingham 2014,364,14.6,28.0,0.52,.215,.346,.397,.743,.182,3.8,...,Josh Willingham,LF,39,Retired,Royals,--,,NR,--,Josh Willingham 2014
Lance Berkman 2013,294,12.9,17.7,0.73,.242,.340,.359,.700,.117,1.7,...,Lance Berkman,DH,42,Retired,Rangers,--,,NR,--,Lance Berkman 2013
Mark DeRosa 2013,236,11.9,20.8,0.57,.235,.326,.407,.733,.172,2.0,...,Mark DeRosa,2B,43,Retired,Blue Jays,--,,NR,--,Mark DeRosa 2013


In [98]:
#Removing those retired players seen above
master_df= master_df.drop(retired_players.index)

In [99]:
list(master_df[master_df.DOLLARS=="Minor Lg"]["PLAYER_YEAR"])
#None now, we should have all numbers in this column

[]

In [100]:
'''
Now convert all numbers available to numeric
'''
master_df = master_df.apply(pd.to_numeric,errors='ignore')
master_df.dtypes

PA_x             int64
BB%            float64
K%             float64
BB/K           float64
AVG_x          float64
OBP            float64
SLG            float64
OPS            float64
ISO            float64
Spd            float64
BABIP          float64
UBR            float64
wGDP           float64
wSB            float64
wRC              int64
wRAA           float64
wOBA           float64
wRC+             int64
G                int64
AB               int64
PA_y             int64
H                int64
1B               int64
2B               int64
3B               int64
HR               int64
R                int64
RBI              int64
BB               int64
IBB              int64
SO               int64
HBP              int64
SF               int64
SH               int64
GDP              int64
SB               int64
CS               int64
AVG_y          float64
PLAYER          object
POS             object
AGE              int64
STATUS          object
OLD_TEAM        object
NEW_TEAM   

In [101]:
master_df[["BB%","K%"]].head()

,BB%,K%
PLAYER_YEAR,,
Jim Thome 2010,17.6,24.1
J.D. Martinez 2017,10.8,26.2
Victor Martinez 2014,10.9,6.6
Franklin Gutierrez 2015,7.4,28.6
Paul Konerko 2010,11.4,17.4


In [102]:
#Save progress
with open('master_df.pkl', 'wb') as picklefile:
    pickle.dump(master_df, picklefile)

In [103]:
#Let's Make a $/Year Column - contract values need to be yearly to be standard
master_df["DOLLARS_PER_YEAR"]=master_df["DOLLARS"]/master_df["YRS"]

In [104]:
master_df.sort_values(by="DOLLARS_PER_YEAR",ascending=False).head(10) 
#This Looks Good
#Yo,Pujols, Cano, Hamilton all signed MONSTER deals so this looks right

,PA_x,BB%,K%,BB/K,AVG_x,OBP,SLG,OPS,ISO,Spd,...,POS,AGE,STATUS,OLD_TEAM,NEW_TEAM,YRS,RK,DOLLARS,PLAYER_YEAR,DOLLARS_PER_YEAR
PLAYER_YEAR,,,,,,,,,,,,,,,,,,,,,
Yoenis Cespedes 2016,543,9.4,19.9,0.47,0.280,0.354,0.530,0.884,0.251,2.9,...,LF,32,Signed,Mets,Mets,4.0,1,110000000.0,Yoenis Cespedes 2016,2.750000e+07
Albert Pujols 2011,651,9.4,8.9,1.05,0.299,0.366,0.541,0.906,0.242,3.9,...,DH,38,Signed (A),Cardinals,Angels,10.0,2,250000000.0,Albert Pujols 2011,2.500000e+07
Yoenis Cespedes 2015,676,4.9,20.9,0.23,0.291,0.328,0.542,0.870,0.251,5.1,...,LF,32,Signed,Mets,Mets,3.0,5,75000000.0,Yoenis Cespedes 2015,2.500000e+07
Josh Hamilton 2012,636,9.4,25.5,0.37,0.285,0.354,0.577,0.930,0.292,4.0,...,LF,36,Signed,Rangers,Angels,5.0,3,123000000.0,Josh Hamilton 2012,2.460000e+07
Robinson Cano 2013,681,9.5,12.5,0.76,0.314,0.383,0.516,0.899,0.202,2.9,...,2B,35,Signed,Yankees,Mariners,10.0,1,240000000.0,Robinson Cano 2013,2.400000e+07
Prince Fielder 2011,692,15.5,15.3,1.01,0.299,0.415,0.566,0.981,0.267,1.9,...,DH,33,Signed (A),Brewers,Tigers,9.0,1,214000000.0,Prince Fielder 2011,2.377778e+07
Jason Heyward 2015,610,9.2,14.8,0.62,0.293,0.359,0.439,0.797,0.146,6.0,...,RF,28,Signed,Cardinals,Cubs,8.0,1,184000000.0,Jason Heyward 2015,2.300000e+07
Chris Davis 2015,670,12.5,31.0,0.40,0.262,0.361,0.562,0.923,0.300,1.9,...,1B,32,Signed,Orioles,Orioles,7.0,7,161000000.0,Chris Davis 2015,2.300000e+07
Justin Upton 2015,620,11.0,25.6,0.43,0.251,0.336,0.454,0.790,0.203,5.7,...,LF,30,Signed,Padres,Tigers,6.0,4,132750000.0,Justin Upton 2015,2.212500e+07


In [105]:
master_df.dtypes #Looks Good

PA_x                  int64
BB%                 float64
K%                  float64
BB/K                float64
AVG_x               float64
OBP                 float64
SLG                 float64
OPS                 float64
ISO                 float64
Spd                 float64
BABIP               float64
UBR                 float64
wGDP                float64
wSB                 float64
wRC                   int64
wRAA                float64
wOBA                float64
wRC+                  int64
G                     int64
AB                    int64
PA_y                  int64
H                     int64
1B                    int64
2B                    int64
3B                    int64
HR                    int64
R                     int64
RBI                   int64
BB                    int64
IBB                   int64
SO                    int64
HBP                   int64
SF                    int64
SH                    int64
GDP                   int64
SB                  

In [106]:
#Save progress as master_df
with open('master_df.pkl', 'wb') as picklefile:
    pickle.dump(master_df, picklefile)